In [ ]:
import sys
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

# Add project root to path
sys.path.append(os.path.abspath('..'))
print("Libraries Imported")

In [ ]:
from src.inversion.forward_model import compute_hyperbolic_phase

In [ ]:
from src.inversion.forward_model import wrap_phase

In [ ]:
from src.inversion.forward_model import get_2channel_representation

def phase_to_channels(phase):
    # Returns (..., H, W, 2) [cos, sin]
    return get_2channel_representation(phase)

In [ ]:
def channels_to_phase(channels):
    # Channels: (..., 2) where [0]=cos, [1]=sin
    # Returns wrapped phase in [-pi, pi]
    cos_val = channels[..., 0]
    sin_val = channels[..., 1]
    if isinstance(channels, torch.Tensor):
        return torch.atan2(sin_val, cos_val)
    else:
        return np.arctan2(sin_val, cos_val)

In [ ]:
# === USER INPUT ===
xc = 0.0          # Center X (microns)
yc = 0.0          # Center Y (microns)
fov = 20.0        # Field of View (degrees)
wavelength = 0.6  # Wavelength (microns)
focal_length = 50.0 # Focal Length (microns)
N = 256           # Resolution
window_size = 100.0 # Window Size (microns)

# 1. Create Grid
grid = np.linspace(-0.5, 0.5, N)
grid_y, grid_x = np.meshgrid(grid, grid, indexing='ij')

X_phys = xc + window_size * grid_x
Y_phys = yc + window_size * grid_y

# 2. Compute Unwrapped Phase
phi_unwrapped = compute_hyperbolic_phase(X_phys, Y_phys, focal_length, wavelength, theta=fov)

# 3. Wrap Phase
phi_wrapped = wrap_phase(phi_unwrapped)

# 4. Convert to Channels
channels = phase_to_channels(phi_wrapped)

# 5. Reconstruct from Channels
phi_reconstructed = channels_to_phase(channels)

# 6. Plot
plt.figure(figsize=(8, 8))
plt.imshow(phi_reconstructed, cmap='jet', vmin=-np.pi, vmax=np.pi)
plt.colorbar(label='Phase (rad)')
plt.title(f"Phase Map (Wrapped)\nparams: xc={xc}, yc={yc}, fov={fov}, wl={wavelength}, f={focal_length}")
plt.axis('off')
plt.show()